In [1]:
import requests
import re
import json
from bs4 import BeautifulSoup
from io import StringIO
import csv
import pandas as pd
import datetime

In [2]:
URL = 'https://finance.yahoo.com/quote/{}/history?p={}'
stock = 'AAPL'
response = requests.get(URL.format(stock, stock))
soup = BeautifulSoup(response.text, 'html.parser')

In [3]:
pattern = re.compile(r'\s--\sData\s--\s')
script_data = soup.find('script', text=pattern).contents[0]    #1st item of the list

In [4]:
#the beginning
script_data[:500]

'\n(function (root) {\n/* -- Data -- */\nroot.App || (root.App = {});\nroot.App.now = 1618829391501;\nroot.App.main = {"context":{"dispatcher":{"stores":{"PageStore":{"currentPageName":"quote","currentEvent":{"eventName":"NEW_PAGE_SUCCESS"},"currentRenderTargetId":"default","pagesConfigRaw":{"base":{"quote":{"layout":{"bundleName":"yahoodotcom-layout.TwoColumnLayout","name":"TwoColumnLayout","config":{"enableHeaderCollapse":true,"additionalBodyWrapperClasses":"Bgc($layoutBgColor)!","contentWrapperClas'

In [5]:
#the end
script_data[-500:]

'put":{"strings":1},"tdv2-applet-sponsored-moments":{"strings":1},"tdv2-applet-stream":{"strings":1},"tdv2-applet-stream-hero":{"strings":1},"tdv2-applet-swisschamp":{"strings":1},"tdv2-applet-uh":{"strings":1},"tdv2-applet-userintent":{"strings":1},"tdv2-applet-video-lightbox":{"strings":1},"tdv2-applet-video-modal":{"strings":1},"tdv2-wafer-adfeedback":{"strings":1},"tdv2-wafer-header":{"strings":1},"yahoodotcom-layout":{"strings":1}}},"options":{"defaultBundle":"td-app-finance"}}}};\n}(this));\n'

In [6]:
start = script_data.find("context")-2

In [7]:
json_data = json.loads(script_data[start:-12])

In [8]:
json_data['context'].keys()

dict_keys(['dispatcher', 'options', 'plugins'])

In [9]:
json_data['context']['dispatcher'].keys()

dict_keys(['stores'])

In [10]:
# json_data['context']['dispatcher']['stores']['RecommendationStore']['recommendedSymbols']['AAPL'][0].get('sourceInterval')

In [11]:
historical_data = json_data['context']['dispatcher']['stores']['HistoricalPriceStore']['prices']
historical_data[0]

{'date': 1618579800,
 'open': 134.3000030517578,
 'high': 134.6699981689453,
 'low': 133.27999877929688,
 'close': 134.16000366210938,
 'volume': 84818500,
 'adjclose': 134.16000366210938}

In [12]:
data = []
keys = historical_data[0].keys()
for i in range(len(historical_data)):
    if historical_data[i].keys() == keys:
        data.append(list(historical_data[i].values()))
data[:2]        

[[1618579800,
  134.3000030517578,
  134.6699981689453,
  133.27999877929688,
  134.16000366210938,
  84818500,
  134.16000366210938],
 [1618493400,
  133.82000732421875,
  135,
  133.63999938964844,
  134.5,
  89347100,
  134.5]]

In [13]:
data = pd.DataFrame(data, columns=list(keys))
data.head()

,date,open,high,low,close,volume,adjclose
0,1618579800,134.300003,134.669998,133.279999,134.160004,84818500,134.160004
1,1618493400,133.820007,135.000000,133.639999,134.500000,89347100,134.500000
2,1618407000,134.940002,135.000000,131.660004,132.029999,87222800,132.029999
3,1618320600,132.440002,134.660004,131.929993,134.429993,91266500,134.429993
4,1618234200,132.520004,132.850006,130.630005,131.240005,91420000,131.240005


In [14]:
date_convert = []
for i in range(len(data)):
    timestamp = datetime.datetime.fromtimestamp(data['date'][i])
    date_convert.append(timestamp.strftime('%Y-%m-%d'))
    
data['date'] = date_convert
data.head()

,date,open,high,low,close,volume,adjclose
0,2021-04-16,134.300003,134.669998,133.279999,134.160004,84818500,134.160004
1,2021-04-15,133.820007,135.000000,133.639999,134.500000,89347100,134.500000
2,2021-04-14,134.940002,135.000000,131.660004,132.029999,87222800,132.029999
3,2021-04-13,132.440002,134.660004,131.929993,134.429993,91266500,134.429993
4,2021-04-12,132.520004,132.850006,130.630005,131.240005,91420000,131.240005


In [15]:
stock_url = 'https://query1.finance.yahoo.com/v7/finance/download/{}?'
params = {
    'range' : '5y',
    'interval' : '1d',
    'events' : 'history' ,
    'includeAdjustedClose' : 'true'
}
response = requests.get(stock_url.format(stock), params=params)

In [16]:
file = StringIO(response.text)
reader = csv.reader(file)
data = list(reader)
column = data[:1]
df = []
for row in data[1:]:
    df.append(row)
data = pd.DataFrame(df, columns=column)
data.head()

,Date,Open,High,Low,Close,Adj Close,Volume
0,2016-04-18,27.222500,27.237499,26.735001,26.870001,24.992023,243286000
1,2016-04-19,26.969999,27.000000,26.557501,26.727501,24.859484,129539600
2,2016-04-20,26.660000,27.022499,26.514999,26.782499,24.910643,122444000
3,2016-04-21,26.732500,26.732500,26.379999,26.492500,24.640911,126210000
4,2016-04-22,26.252501,26.620001,26.155001,26.420000,24.573479,134732400


In [17]:
data.to_csv("Stock_price.csv",index=False)